In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv(r"SBS_Processed_Datasets/02_cleaned_output.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8107 entries, 0 to 8106
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             8107 non-null   object 
 1   transmission     8107 non-null   object 
 2   owner_no         8107 non-null   int64  
 3   brand            8107 non-null   object 
 4   model            8107 non-null   object 
 5   model_year       8107 non-null   int64  
 6   variant_name     8107 non-null   object 
 7   price            8107 non-null   float64
 8   registered_year  8107 non-null   int64  
 9   fuel_type        8107 non-null   object 
 10  kms_driven       8107 non-null   float64
 11  mileage_kmpl     8107 non-null   float64
 12  engine_cc        8107 non-null   int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 823.5+ KB


In [4]:
df.head()

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,mileage_kmpl,engine_cc
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,120000.0,23.10,998
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,32706.0,17.00,1497
2,bangalore,manual,1,tata,tata tiago,2018,1.2 revotron xz,585000.0,2018,petrol,11949.0,23.84,1199
3,bangalore,manual,1,hyundai,hyundai xcent,2014,1.2 kappa s option,462000.0,2014,petrol,17794.0,19.10,1197
4,bangalore,manual,1,maruti,maruti sx4 s cross,2015,ddis 200 zeta,790000.0,2015,diesel,60000.0,23.65,1248


FEATURE ENGINEERING

In [5]:
#STEP_1 ADD AGE RELATED FEATURES

# Get the current year
current_year = datetime.now().year

# Calculate car age
df['car_age'] = current_year - df['registered_year']
df['model_age'] = current_year - df['model_year']
df['registration_lag']  = df['registered_year'] - df['model_year'] 
df['normalized_model_age']  = df['car_age'] / (df['model_age']  + 1)

In [6]:
#STEP_2 Add price_per_km

bins = [0, 5000, 10000, 50000, 100000, float('inf')]
labels = ['Unused', 'Very_Low', 'Low', 'Moderate', 'High']  # Remove one label to match bin intervals
df['kms_bins'] = pd.cut(df['kms_driven'], bins=bins, labels=labels, include_lowest=True)

In [7]:
#STEP_3 Create mileage_normalized

# Calculate normalized mileage
df['mileage_normalized'] = df['mileage_kmpl'] / df['mileage_kmpl'].max()

In [8]:
#STEP_4 High Mileage Flag

# high_mileage (binary feature)
high_mileage_threshold = 150000  # Define the threshold for high mileage
df['high_mileage'] = (df['kms_driven'] > high_mileage_threshold).astype(int)

In [9]:
#STEP_5 Multiple Owners

# Flag for cars with multiple previous owners
df['multiple_owners'] = (df['owner_no'] > 1).astype(int)

In [10]:
#STEP_6 Brand Popularity

# Compute average price per brand
brand_avg_price = df.groupby('brand')['price'].mean().to_dict()

# Map the average price to each row
df['brand_popularity'] = df['brand'].map(brand_avg_price)

# Example: Adding brand popularity column
print(df[['brand', 'brand_popularity']].head())

     brand  brand_popularity
0   maruti     504517.464043
1     ford     730169.934641
2     tata     748643.178049
3  hyundai     595115.553404
4   maruti     504517.464043


In [11]:
#STEP_7 kms_per_year
df['kms_per_year'] = df['kms_driven'] / (df['car_age'] + 1)

In [12]:
df.head(2)

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,...,car_age,model_age,registration_lag,normalized_model_age,kms_bins,mileage_normalized,high_mileage,multiple_owners,brand_popularity,kms_per_year
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,...,9,9,0,0.900000,High,0.648876,0,1,504517.464043,12000.000000
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,...,6,6,0,0.857143,Low,0.477528,0,1,730169.934641,4672.285714


In [13]:
df.describe()

,owner_no,model_year,price,registered_year,kms_driven,mileage_kmpl,engine_cc,car_age,model_age,registration_lag,normalized_model_age,mileage_normalized,high_mileage,multiple_owners,brand_popularity,kms_per_year
count,8107.000000,8107.000000,8.107000e+03,8107.000000,8.107000e+03,8107.000000,8107.000000,8107.000000,8107.000000,8107.000000,8107.000000,8107.000000,8107.000000,8107.00000,8.107000e+03,8107.000000
mean,1.364870,2016.450475,8.854950e+05,2016.518071,5.956622e+04,19.190692,1422.127421,7.481929,7.549525,0.067596,0.844578,0.539064,0.007648,0.29493,8.854950e+05,7298.162929
std,0.635211,3.853358,9.890027e+05,3.862395,7.492185e+04,3.671887,465.210269,3.862395,3.853358,0.319016,0.088946,0.103143,0.087122,0.45604,7.404078e+05,10446.455266
min,1.000000,1985.000000,2.800000e+04,1985.000000,1.010000e+02,7.080000,624.000000,1.000000,1.000000,0.000000,0.105263,0.198876,0.000000,0.00000,7.333333e+04,50.500000
25%,1.000000,2014.000000,3.980000e+05,2014.000000,3.121800e+04,17.000000,1197.000000,5.000000,5.000000,0.000000,0.800000,0.477528,0.000000,0.00000,5.045175e+05,4506.570707
50%,1.000000,2017.000000,6.000000e+05,2017.000000,5.430500e+04,18.900000,1248.000000,7.000000,7.000000,0.000000,0.875000,0.530899,0.000000,0.00000,5.951156e+05,6547.000000
75%,2.000000,2019.000000,9.250000e+05,2019.000000,8.000000e+04,21.400000,1498.000000,10.000000,10.000000,0.000000,0.909091,0.601124,0.000000,1.00000,7.486432e+05,9090.909091
max,5.000000,2023.000000,9.600000e+06,2023.000000,5.500000e+06,35.600000,5000.000000,39.000000,39.000000,16.000000,0.975000,1.000000,1.000000,1.00000,6.343750e+06,785714.285714


In [14]:
df.head(2)

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,...,car_age,model_age,registration_lag,normalized_model_age,kms_bins,mileage_normalized,high_mileage,multiple_owners,brand_popularity,kms_per_year
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,...,9,9,0,0.900000,High,0.648876,0,1,504517.464043,12000.000000
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,...,6,6,0,0.857143,Low,0.477528,0,1,730169.934641,4672.285714


In [15]:
df.columns

Index(['city', 'transmission', 'owner_no', 'brand', 'model', 'model_year',
       'variant_name', 'price', 'registered_year', 'fuel_type', 'kms_driven',
       'mileage_kmpl', 'engine_cc', 'car_age', 'model_age', 'registration_lag',
       'normalized_model_age', 'kms_bins', 'mileage_normalized',
       'high_mileage', 'multiple_owners', 'brand_popularity', 'kms_per_year'],
      dtype='object')

In [16]:
# Export it
df.to_csv(r"SBS_Processed_Datasets\03_feature_engineered_output.csv",index=False)